In [38]:
# Import ML libraries
import pycaret
import xgboost

# Snowpark for Python
from snowflake.snowpark.session import Session
import snowflake.snowpark.types as T
import snowflake.snowpark.functions as F
import snowflake.snowpark as snowpark
from snowflake.snowpark.functions import datediff, to_date, col, expr

# Import Misc
import json
import pandas as pd
# from pycaret.classification import setup, compare_models


In [39]:
# Create Snowflake Session object
connection_parameters = json.load(open("connection.json"))
session = Session.builder.configs(connection_parameters).create()

In [40]:
# Connecting to the correct table
tableName = 'PURCHASE_ORDER_HISTORY'
dataframe = session.table(tableName)

# Calculation to find the lag between Planned Delivery from Actual Delivery
dataframe = dataframe.withColumn("target_feature",
                                    datediff('day', 
                                            col("DELIVERY_DATE_ML"), 
                                            col("FIRST_GR_POSTING_DATE_ML")))


# Example: Selecting specific columns
# This selects only a subset of columns. Adjust the column names as needed.
filtered_dataframe = dataframe.select(
    col("PURCHASE_DOCUMENT_ITEM_ID"), # ID for purchase order
    col("CREATE_DATE"),            # day purchase order was created
    col("COMPANY_CODE_ID"),           # copmany w/in INVISTA making purchase
    col("VENDOR_ID"),                 # ID of the vendor "we" are purchasing from
    col("POSTAL_CD"),                 # postal code associated w company code ID
    col("MATERIAL_ID"),               # ID of material being purchase
    col("SUB_COMMODITY_DESC"),        # description of sub commodity
    col("MRP_TYPE_ID"),               # determined if material is reordered manually or automatically
    col("PLANT_ID"),                  # ID of plant making purchase
    col("REQUESTED_DELIVERY_DATE"),# delivery date from requisition
    col("INBOUND_DELIVERY_ID"),       # ID for delivery
    col("INBOUND_DELIVERY_ITEM_ID"),  # ID of item w/in delivery
    col("PLANNED_DELIVERY_DAYS"),     # Amount of days expected to take
    col("FIRST_GR_POSTING_DATE"),  # expected delivery date        
    col("target_feature")             # Lag between Planned Delivery from Actual Delivery 
)


# Print a sample of the filtered dataframe to standard output.
filtered_dataframe.show()

# Optionally, you might want to filter rows based on some conditions
# Example: Filtering out rows where FIRST_GR_POSTING_DATE_ML is NULL
filtered_dataframe = filtered_dataframe.filter(col("FIRST_GR_POSTING_DATE").is_not_null())

# filtered_dataframe = filtered_dataframe[filtered_dataframe['PLANNED_DELIVERY_DAYS'] < 6]

# Show the DataFrame after filtering
filtered_dataframe.show()

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"PURCHASE_DOCUMENT_ITEM_ID"  |"CREATE_DATE"  |"COMPANY_CODE_ID"  |"VENDOR_ID"  |"POSTAL_CD"  |"MATERIAL_ID"  |"SUB_COMMODITY_DESC"                    |"MRP_TYPE_ID"  |"PLANT_ID"  |"REQUESTED_DELIVERY_DATE"  |"INBOUND_DELIVERY_ID"  |"INBOUND_DELIVERY_ITEM_ID"  |"PLANNED_DELIVERY_DAYS"  |"FIRST_GR_POSTING_DATE"  |"TARGET_FEATURE"  |
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [41]:
# Assuming 'filtered_dataframe' is the DataFrame you've prepared in Snowflake
# Convert the Snowpark DataFrame to a Pandas DataFrame with consideration for NULL values

# Convert DataFrame to Pandas, handling NULL values by allowing float conversion
df = filtered_dataframe.fillna(0).to_pandas()  # This replaces NULL with 0 before conversion

Input value type doesn't match the target column data type, this replacement was skipped. Column Name: "COMPANY_CODE_ID", Type: StringType(16777216), Input Value: 0, Type: <class 'int'>
Input value type doesn't match the target column data type, this replacement was skipped. Column Name: "VENDOR_ID", Type: StringType(16777216), Input Value: 0, Type: <class 'int'>
Input value type doesn't match the target column data type, this replacement was skipped. Column Name: "POSTAL_CD", Type: StringType(16777216), Input Value: 0, Type: <class 'int'>
Input value type doesn't match the target column data type, this replacement was skipped. Column Name: "SUB_COMMODITY_DESC", Type: StringType(16777216), Input Value: 0, Type: <class 'int'>
Input value type doesn't match the target column data type, this replacement was skipped. Column Name: "PLANNED_DELIVERY_DAYS", Type: StringType(16777216), Input Value: 0, Type: <class 'int'>


In [42]:
df.head()

,PURCHASE_DOCUMENT_ITEM_ID,CREATE_DATE,COMPANY_CODE_ID,VENDOR_ID,POSTAL_CD,MATERIAL_ID,SUB_COMMODITY_DESC,MRP_TYPE_ID,PLANT_ID,REQUESTED_DELIVERY_DATE,INBOUND_DELIVERY_ID,INBOUND_DELIVERY_ITEM_ID,PLANNED_DELIVERY_DAYS,FIRST_GR_POSTING_DATE,TARGET_FEATURE
0,10,20180907,CN15,V4014,29078,2100007708,Custom Manufacturing,1,1016,20181116,183615169,900001,52.0,20181122,6
1,20,20180907,CN20,8010095928,201206,0,Tanks and Process Equipment,0,1032,20180928,0,0,0.0,0,0
2,10,20180907,CA10,8010003146,L6L 6R2,1100125572,Piping & Tubing,1,4036,20181001,0,0,24.0,20181205,23
3,180,20180907,CA10,8010005836,N2C 0B7,0,Material Handling,0,4036,20180908,0,0,0.0,20180918,10
4,60,20180907,CA10,8010005836,N2C 0B7,0,Material Handling,0,4036,20180908,0,0,0.0,20180918,10


In [43]:
df['TARGET_FEATURE']

0           6
1           0
2          23
3          10
4          10
           ..
1139387    -2
1139388    -1
1139389     0
1139390    -1
1139391   -27
Name: TARGET_FEATURE, Length: 1139392, dtype: int32

In [44]:
import numpy as np
import torch


In [45]:

# Handle missing values
# data.fillna(0, inplace=True)  

# Impute missing SUB_COMMODITY_DESC
df['SUB_COMMODITY_DESC'].fillna('Unknown', inplace=True)

# Convert categorical columns to one-hot encoding
df = pd.get_dummies(df, columns=['SUB_COMMODITY_DESC'])

# from sklearn.preprocessing import OneHotEncoder

# encoder = OneHotEncoder(sparse=False)  # Dense output
# encoded_data = encoder.fit_transform(df[['SUB_COMMODITY_DESC']])


In [46]:
import pandas as pd
import re

def clean_delivery_days(value):
    if isinstance(value, str):
        # Remove leading/trailing whitespace
        value = value.strip() 

        # Check for timestamp format and handle separately
        if re.match(r"\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}", value):  
            return ""  # Replace timestamps with NA or another placeholder
        else:
            return value
    else:
        return value

df['PLANNED_DELIVERY_DAYS'] = df['PLANNED_DELIVERY_DAYS'].apply(clean_delivery_days)


In [47]:
# Hash Alphanumeric columns
df['VENDOR_ID'] = df['VENDOR_ID'].apply(hash)
df['POSTAL_CD'] = df['POSTAL_CD'].apply(hash)
df['COMPANY_CODE_ID'] = df['COMPANY_CODE_ID'].apply(hash)




In [48]:
def remove_decimal(value):
    return value.split('.')[0]  # Split by the decimal and keep the integer part

df['PLANNED_DELIVERY_DAYS'] = df['PLANNED_DELIVERY_DAYS'].apply(remove_decimal)

for value in df['PLANNED_DELIVERY_DAYS'].unique():
    print(value, type(value), repr(value)) 



AttributeError: 'int' object has no attribute 'split'

In [ ]:
def convert_to_numeric(col):
    try:
        df[col] = pd.to_numeric(df[col])
    except ValueError:
        print(f"Error converting column '{col}': Contains non-numeric values")

convert_to_numeric('PURCHASE_DOCUMENT_ITEM_ID')
convert_to_numeric('CREATE_DATE')
convert_to_numeric('VENDOR_ID')
convert_to_numeric('POSTAL_CD')
convert_to_numeric('MATERIAL_ID')
convert_to_numeric('MRP_TYPE_ID')
convert_to_numeric('PLANT_ID')
convert_to_numeric('INBOUND_DELIVERY_ID')
convert_to_numeric('INBOUND_DELIVERY_ITEM_ID')
convert_to_numeric('PLANNED_DELIVERY_DAYS')
convert_to_numeric('FIRST_GR_POSTING_DATE')


Error converting column 'PLANNED_DELIVERY_DAYS': Contains non-numeric values


In [ ]:
# Display the dataframe
df.head()


,PURCHASE_DOCUMENT_ITEM_ID,CREATE_DATE,COMPANY_CODE_ID,VENDOR_ID,POSTAL_CD,MATERIAL_ID,MRP_TYPE_ID,PLANT_ID,REQUESTED_DELIVERY_DATE,INBOUND_DELIVERY_ID,...,SUB_COMMODITY_DESC_Tools,SUB_COMMODITY_DESC_Transport operations services,"SUB_COMMODITY_DESC_Transportation, Storage, Mail Services",SUB_COMMODITY_DESC_Travel Services,SUB_COMMODITY_DESC_Tubes & Cores,SUB_COMMODITY_DESC_Unknown,SUB_COMMODITY_DESC_Valves,SUB_COMMODITY_DESC_Vehicles,SUB_COMMODITY_DESC_Waste Disposal Services,SUB_COMMODITY_DESC_Water Treatment Chemicals
0,10,20180907,7674034914737855841,2102158588121848955,4711035905609964672,2100007708,1,1016,20181116,183615169,...,0,0,0,0,0,0,0,0,0,0
1,20,20180907,5599762719532389283,745970689818259105,-481906218548842100,0,0,1032,20180928,0,...,0,0,0,0,0,0,0,0,0,0
2,10,20180907,-7242613046748319157,7278734403367281877,-2233222475004576134,1100125572,1,4036,20181001,0,...,0,0,0,0,0,0,0,0,0,0
3,180,20180907,-7242613046748319157,-6001814796187456735,6917799730438150654,0,0,4036,20180908,0,...,0,0,0,0,0,0,0,0,0,0
4,60,20180907,-7242613046748319157,-6001814796187456735,6917799730438150654,0,0,4036,20180908,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:

# Show types of each column
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1139392 entries, 0 to 1139391
Data columns (total 87 columns):
 #   Column                                                     Non-Null Count    Dtype 
---  ------                                                     --------------    ----- 
 0   PURCHASE_DOCUMENT_ITEM_ID                                  1139392 non-null  int16 
 1   CREATE_DATE                                                1139392 non-null  int32 
 2   COMPANY_CODE_ID                                            1139392 non-null  int64 
 3   VENDOR_ID                                                  1139392 non-null  int64 
 4   POSTAL_CD                                                  1139392 non-null  int64 
 5   MATERIAL_ID                                                1139392 non-null  int64 
 6   MRP_TYPE_ID                                                1139392 non-null  int8  
 7   PLANT_ID                                                   1139392 non-null  int1

In [ ]:

# Create feature & target tensors
features = df.drop('TARGET_FEATURE', axis=1)
targets = df['TARGET_FEATURE']
X = torch.tensor(features.values.astype(np.float32))
y = torch.tensor(targets.values.astype(np.float32))


ValueError: could not convert string to float: '2023-05-30T23:07:43'